In [1]:
import time
import os
import re
import sys
import numpy as np
#import cupy as cp
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import hilbert, chirp
from plotly.offline import plot
import plotly.express as px     
#from Utils2 import *
from Utils99 import *
from Utils_graph import *
import pickle as pkl
import re
#from dsp99 import *
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
#from scipy.stats import norm, kurtosis
from scipy.signal import stft
from scipy import signal
#from sktime.libs.vmdpy import VMD
from vmdpyR2 import * 
from collections import Counter
from scipy.stats import kurtosis
from scipy.signal import butter, filtfilt
from vmdpyR2 import * 
import nbformat
window = 1024
fs = 25600

'''
Rotação: 2100, BPFO (Outer Race): 107.91 Hz
Rotação: 2100, BPFI (Inner Race): 172.09 Hz
Rotação: 2100, BSF (Ball Spin): 72.33 Hz
Rotação: 2100, FTF (Cage Frequency): 13.49 Hz

Rotação: 2250, BPFO (Outer Race): 115.62 Hz
Rotação: 2250, BPFI (Inner Race): 184.38 Hz
Rotação: 2250, BSF (Ball Spin): 77.50 Hz
Rotação: 2250, FTF (Cage Frequency): 14.45 Hz

Rotação: 2400, BPFO (Outer Race): 123.32 Hz
Rotação: 2400, BPFI (Inner Race): 196.68 Hz
Rotação: 2400, BSF (Ball Spin): 82.66 Hz
Rotação: 2400, FTF (Cage Frequency): 15.42 Hz
'''

'\nRotação: 2100, BPFO (Outer Race): 107.91 Hz\nRotação: 2100, BPFI (Inner Race): 172.09 Hz\nRotação: 2100, BSF (Ball Spin): 72.33 Hz\nRotação: 2100, FTF (Cage Frequency): 13.49 Hz\n\nRotação: 2250, BPFO (Outer Race): 115.62 Hz\nRotação: 2250, BPFI (Inner Race): 184.38 Hz\nRotação: 2250, BSF (Ball Spin): 77.50 Hz\nRotação: 2250, FTF (Cage Frequency): 14.45 Hz\n\nRotação: 2400, BPFO (Outer Race): 123.32 Hz\nRotação: 2400, BPFI (Inner Race): 196.68 Hz\nRotação: 2400, BSF (Ball Spin): 82.66 Hz\nRotação: 2400, FTF (Cage Frequency): 15.42 Hz\n'

In [2]:
class VMD_opt:
	def __init__(self):
		VMD_opt.k = 0
		VMD_opt.kmax = 0
		VMD_opt.signal = 0
		VMD_opt.components = 0

class VMD_Mode:
	def __init__(self,k):
		VMD_Mode.k = k
		VMD_Mode.max_krts = 0
		VMD_Mode.signal = 0
		VMD_Mode.components = 0
		VMD_Mode.freq = 0
		VMD_Mode.components_psd = 0
		VMD_Mode.components_ent = 0

In [ ]:
def calculate_k_modes(sig,max_modes = 5):
    k_modes = range(2,max_modes)
    vmd_opt = VMD_Mode(1)
    for k in (k_modes):
        #print(k)
        vmd_mode = VMD_Mode(k)
        vmd_mode.signal = sig
        max_kur = 0
        modes, _, _ = VMD(sig, alpha=2000, tau=0, K=k, DC=0, init=1, tol=1e-6)
        for mode in modes:
            env = np.abs(hilbert(mode))
            env = env - np.mean(env)
            kur = kurtosis(env, fisher=False)
            if kur > max_kur:
                max_kur = kur
        #print('max_kur',max_kur, 'k:', k)
        vmd_mode.components = np.concatenate(([sig],modes),axis = 0)
        vmd_mode.max_krts = max_kur
        vmd_mode.k = k

        if vmd_mode.max_krts > vmd_opt.max_krts:
            vmd_opt = vmd_mode
        k_max = vmd_opt.k
        components = vmd_opt.components
        #print('vmd_opt.max_krts:',vmd_opt.max_krts, 'vmd_opt.k:',vmd_opt.k)
 
    return k_max, components


def bandpass_filter(signal_data, lowcut, highcut, fs, order=4):
    """
    Aplica um filtro passa-banda em um sinal.

    Parâmetros:
    - signal_data: np.array -> Sinal de entrada
    - lowcut: float -> Frequência de corte inferior (Hz)
    - highcut: float -> Frequência de corte superior (Hz)
    - fs: float -> Frequência de amostragem (Hz)
    - order: int -> Ordem do filtro (padrão: 4)

    Retorna:
    - Sinal filtrado
    """

    nyquist = 0.5 * fs  # Frequência de Nyquist
    low = lowcut / nyquist
    high = highcut / nyquist

    # Criando filtro Butterworth passa-banda
    b, a = signal.butter(order, [low, high], btype='band')

    # Aplicando filtro no sinal
    filtered_signal = signal.filtfilt(b, a, signal_data)

    return filtered_signal

In [2]:
path = '03_VMDY/'
brngs = os.listdir(path)
k = 10
for brng in brngs:
    if brng.startswith('Bearing3_5'):
        samples = os.listdir(path+brng)
        samples.sort(key=lambda x: int(re.search(r'\d+', x).group()))
        for sample in samples:
            print(path+brng+'/'+sample)
            df = pd.read_csv(path+brng+'/'+sample)
            sig1 = df['Vertical_vibration_signals'].values

            modes, _, _ = VMD(sig1, alpha=2000, tau=0, K=k, DC=0, init=1, tol=1e-6)
            names = [f'IMF{i+1}' for i in range(len(modes))]
            modes=np.concatenate(([sig1],modes),axis=0)
            names = ['SIG'] + names
            
            df = pd.DataFrame(modes).T
            df.columns = names
            df.to_csv(path+brng+'/'+sample, index = False)


03_VMDY/Bearing3_5/1.csv
03_VMDY/Bearing3_5/2.csv
03_VMDY/Bearing3_5/3.csv
03_VMDY/Bearing3_5/4.csv
03_VMDY/Bearing3_5/5.csv
03_VMDY/Bearing3_5/6.csv
03_VMDY/Bearing3_5/7.csv
03_VMDY/Bearing3_5/8.csv
03_VMDY/Bearing3_5/9.csv
03_VMDY/Bearing3_5/10.csv
03_VMDY/Bearing3_5/11.csv
03_VMDY/Bearing3_5/12.csv
03_VMDY/Bearing3_5/13.csv
03_VMDY/Bearing3_5/14.csv
03_VMDY/Bearing3_5/15.csv
03_VMDY/Bearing3_5/16.csv
03_VMDY/Bearing3_5/17.csv
03_VMDY/Bearing3_5/18.csv
03_VMDY/Bearing3_5/19.csv
03_VMDY/Bearing3_5/20.csv
03_VMDY/Bearing3_5/21.csv
03_VMDY/Bearing3_5/22.csv
03_VMDY/Bearing3_5/23.csv
03_VMDY/Bearing3_5/24.csv
03_VMDY/Bearing3_5/25.csv
03_VMDY/Bearing3_5/26.csv
03_VMDY/Bearing3_5/27.csv
03_VMDY/Bearing3_5/28.csv
03_VMDY/Bearing3_5/29.csv
03_VMDY/Bearing3_5/30.csv
03_VMDY/Bearing3_5/31.csv
03_VMDY/Bearing3_5/32.csv
03_VMDY/Bearing3_5/33.csv
03_VMDY/Bearing3_5/34.csv
03_VMDY/Bearing3_5/35.csv
03_VMDY/Bearing3_5/36.csv
03_VMDY/Bearing3_5/37.csv
03_VMDY/Bearing3_5/38.csv
03_VMDY/Bearing3_5/39

In [ ]:
path = '03_VMDY/'
brngs = os.listdir(path)
k = 10
for brng in brngs:
    
    samples = os.listdir(path+brng)
    samples.sort(key=lambda x: int(re.search(r'\d+', x).group()))
    for sample in samples:
        print(path+brng+'/'+sample)
        df = pd.read_csv(path+brng+'/'+sample)
        sig1 = df['Vertical_vibration_signals'].values

        modes, _, _ = VMD(sig1, alpha=2000, tau=0, K=k, DC=0, init=1, tol=1e-6)
        names = [f'IMF{i+1}' for i in range(len(modes))]
        modes=np.concatenate(([sig1],modes),axis=0)
        names = ['SIG'] + names
        
        df = pd.DataFrame(modes).T
        df.columns = names
        df.to_csv(path+brng+'/'+sample, index = False)

03_VMDY/Bearing1_1/1.csv
03_VMDY/Bearing1_1/2.csv
03_VMDY/Bearing1_1/3.csv
03_VMDY/Bearing1_1/4.csv
03_VMDY/Bearing1_1/5.csv
03_VMDY/Bearing1_1/6.csv
03_VMDY/Bearing1_1/7.csv
03_VMDY/Bearing1_1/8.csv
03_VMDY/Bearing1_1/9.csv
03_VMDY/Bearing1_1/10.csv
03_VMDY/Bearing1_1/11.csv
03_VMDY/Bearing1_1/12.csv
03_VMDY/Bearing1_1/13.csv
03_VMDY/Bearing1_1/14.csv
03_VMDY/Bearing1_1/15.csv
03_VMDY/Bearing1_1/16.csv
03_VMDY/Bearing1_1/17.csv
03_VMDY/Bearing1_1/18.csv
03_VMDY/Bearing1_1/19.csv
03_VMDY/Bearing1_1/20.csv
03_VMDY/Bearing1_1/21.csv
03_VMDY/Bearing1_1/22.csv
03_VMDY/Bearing1_1/23.csv
03_VMDY/Bearing1_1/24.csv
03_VMDY/Bearing1_1/25.csv
03_VMDY/Bearing1_1/26.csv
03_VMDY/Bearing1_1/27.csv
03_VMDY/Bearing1_1/28.csv
03_VMDY/Bearing1_1/29.csv
03_VMDY/Bearing1_1/30.csv
03_VMDY/Bearing1_1/31.csv
03_VMDY/Bearing1_1/32.csv
03_VMDY/Bearing1_1/33.csv
03_VMDY/Bearing1_1/34.csv
03_VMDY/Bearing1_1/35.csv
03_VMDY/Bearing1_1/36.csv
03_VMDY/Bearing1_1/37.csv
03_VMDY/Bearing1_1/38.csv
03_VMDY/Bearing1_1/39

In [30]:
path = '00_XJTU/XJTU-SY_Bearing_Datasets/35Hz12kN/Bearing1_1/123.csv'
df = pd.read_csv(path)
#df = moving_average_df(df,3)
sig = df['Horizontal_vibration_signals'].values
modes,_,_ = VMD(sig, alpha=2000, tau=0, K=10, DC=0, init=1, tol=1e-6)


In [40]:
modes1 = (np.concatenate(([sig],modes),axis=0))
names = [f'IMF{i+1}' for i in range(len(modes))]
names = ['SIG'] + names

In [41]:
df = pd.DataFrame(modes1).T
df.columns = names
df

,SIG,IMF1,IMF2,IMF3,IMF4,IMF5,IMF6,IMF7,IMF8,IMF9,IMF10
0,-3.186226,1.990790,-4.145280,-0.396862,0.523187,0.092681,-0.075735,-0.068935,-0.099665,0.025181,0.011317
1,-2.718699,1.948242,-3.839902,-0.337959,0.061665,-0.119469,0.060542,0.128488,0.228942,-0.085754,-0.058506
2,-2.698874,1.853094,-3.263519,-0.244310,-0.493427,-0.156621,0.059901,-0.098297,-0.255428,0.117746,0.097733
3,-1.729584,1.693536,-2.473345,-0.148487,-0.682662,0.087055,-0.075568,-0.049534,0.142987,-0.123185,-0.137861
4,-0.062287,1.462414,-1.538008,-0.075632,-0.316565,0.221790,-0.046897,0.196856,0.067063,0.101376,0.179866
...,...,...,...,...,...,...,...,...,...,...,...
32763,-1.983047,-2.605896,1.330391,0.472167,-0.402664,-0.526411,0.083857,-0.608556,-0.051893,0.600020,0.243357
32764,-9.157157,-3.331524,1.595749,-1.072155,-2.482518,0.023157,-0.808929,0.505224,-0.482229,-0.872840,-0.224534
32765,-4.796529,-3.903093,1.793666,-2.478308,-2.213512,0.476503,0.225031,0.201991,0.753183,0.884862,0.178493
32766,-9.346092,-4.303513,1.918812,-3.550326,0.038418,0.159937,0.629861,-0.749934,-0.691109,-0.667750,-0.129832


In [33]:
names = [f'IMF{i}' for i in range(len(modes))]
df = pd.DataFrame(modes).T
df.columns = names
df

,IMF0,IMF1,IMF2,IMF3,IMF4,IMF5,IMF6,IMF7,IMF8,IMF9
0,1.990790,-4.145280,-0.396862,0.523187,0.092681,-0.075735,-0.068935,-0.099665,0.025181,0.011317
1,1.948242,-3.839902,-0.337959,0.061665,-0.119469,0.060542,0.128488,0.228942,-0.085754,-0.058506
2,1.853094,-3.263519,-0.244310,-0.493427,-0.156621,0.059901,-0.098297,-0.255428,0.117746,0.097733
3,1.693536,-2.473345,-0.148487,-0.682662,0.087055,-0.075568,-0.049534,0.142987,-0.123185,-0.137861
4,1.462414,-1.538008,-0.075632,-0.316565,0.221790,-0.046897,0.196856,0.067063,0.101376,0.179866
...,...,...,...,...,...,...,...,...,...,...
32763,-2.605896,1.330391,0.472167,-0.402664,-0.526411,0.083857,-0.608556,-0.051893,0.600020,0.243357
32764,-3.331524,1.595749,-1.072155,-2.482518,0.023157,-0.808929,0.505224,-0.482229,-0.872840,-0.224534
32765,-3.903093,1.793666,-2.478308,-2.213512,0.476503,0.225031,0.201991,0.753183,0.884862,0.178493
32766,-4.303513,1.918812,-3.550326,0.038418,0.159937,0.629861,-0.749934,-0.691109,-0.667750,-0.129832
